# Singapore HDB Price Analysis (Using SQL)

In [18]:
import pandas as pd
import sqlite3

In [19]:
df = pd.read_csv('HDB_faltData.csv')
df['month'] = pd.to_datetime(df['month'])
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [20]:
conn = sqlite3.connect('flatData.db')
df.to_sql('flat_data', conn, if_exists='replace', index=False)

199101

In [21]:
query = 'select * from flat_data limit 5'
pd.read_sql(query, conn)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01-01 00:00:00,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01-01 00:00:00,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01-01 00:00:00,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01-01 00:00:00,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01-01 00:00:00,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


## Q1: Which top 5 town has the highest average resale price?

In [28]:
query = 'select town, round(avg(resale_price),2) as avg_resale_price from flat_data group by town order by avg_resale_price desc limit 5'
pd.read_sql(query, conn)

,town,avg_resale_price
0,BUKIT TIMAH,753923.87
1,BISHAN,692304.24
2,CENTRAL AREA,674010.11
3,QUEENSTOWN,621550.61
4,BUKIT MERAH,621503.22


## Q2: How do resale prices trend over time?

In [34]:
query = 'select strftime("%Y", month) as year, round(avg(resale_price),2) as avg_flat_price from flat_data group by strftime("%Y", month)'
trend = pd.read_sql(query, conn)
trend

,year,avg_flat_price
0,2017,443888.52
1,2018,441282.06
2,2019,432137.91
3,2020,452279.38
4,2021,511381.24
5,2022,549714.33
6,2023,571804.00
7,2024,612531.92
8,2025,634882.33


In [49]:
price_2017 = trend.iloc[0]['avg_flat_price']
price_2023 = trend.iloc[-2]['avg_flat_price']
price_increase_percent = round(((price_2023 - price_2017) / price_2017) * 100)
print(f'{price_increase_percent}%')

38%


## Q3: Which flat type is most expensive?

In [33]:
query = 'select flat_type, round(avg(resale_price),2) as avg_flat_price from flat_data group by flat_type order by avg_flat_price desc'
pd.read_sql(query, conn)

,flat_type,avg_flat_price
0,MULTI-GENERATION,842086.75
1,EXECUTIVE,712586.93
2,5 ROOM,606448.09
3,4 ROOM,511462.03
4,3 ROOM,359912.46
5,2 ROOM,286231.10
6,1 ROOM,207042.08
